_This notebook is part of the material for the ["An Introduction to Neural Networks"](https://indico.cism.ucl.ac.be/event/106/) session of the [2021 CISM/CÉCI trainings](https://indico.cism.ucl.ac.be/category/6/)._

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ramonpeter/cism-nn2021/blob/main/ann/artificial_neural_networks.ipynb)

# Artificial Neural Networks

Tensorflow is a widely used library for machine learning, especially deep learning, both training and inference (evaluating trained neural networks on new data).
It was developed by the Google Brain team, and is open source software.
On the [website](https://www.tensorflow.org) you will find many libraries and tools for common tasks related to machine learning, and a lot of training material and examples.

In this second part, we will use [Tensorflow](https://www.tensorflow.org) and its high-level API [Keras](https://keras.io) 
on a bank business problem.


### Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

## Part 1 - Data Preprocessing

### Importing the dataset

In [ ]:
dataset = pd.read_csv('https://raw.githubusercontent.com/ramonpeter/cism-nn2021/main/ann/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [ ]:
print(X.shape)
print(X)

In [ ]:
print(y)

### Encoding categorical data

Label Encoding the "Gender" column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [ ]:
print(X)

One Hot Encoding the "Geography" column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X.shape)
print(X)


### Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Sequential API

Adding the input layer and the hidden layers

In [ ]:
ann = tf.keras.Sequential([
    tf.keras.layers.Dense(units=6, activation='relu'),
    tf.keras.layers.Dense(units=6, activation='relu'),
])

Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Functional API

With the functional API multiple number of inputs and outputs are possible as well as non-linear topologies  (e.g. a residual connection, a multi-branch model).  
In this case we explicitly need to define the Input and its shape.

In [ ]:
inputs = tf.keras.Input(shape=(12,))
x = tf.keras.layers.Dense(6, activation="relu")(inputs)
x = tf.keras.layers.Dense(6, activation="relu")(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
f_model = tf.keras.Model(inputs=inputs, outputs=outputs, name="functional_model")

An advantage of this API is, that we can explitcly check the model summary and its graph

In [ ]:
f_model.summary()

## Part 3 - Training the ANN

### Compiling the ANN

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [ ]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

## Part 4 - Predicting the result of a single observation

### Exercise

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

### Solution

In [ ]:
# TODO